## Introduction

In this code, you are simulating a 1D bar that is loaded under tensile loading conditions. The magnitude of the loading that the bar is subject to is determined by the applied displacement value (so displacement control is used). The schematic is presented below:

![1D Bar](1d_bar.png)

To make life easier, I have wrapped the FEM code in a function `run_experiment`. You need to give it some parameters:
- `timesteps`: the number of timesteps used in the experiment.
- `element_count`: the number of elements the mesh is discretised into.
- `lengthscale_parameter`: the phase-field model parameter that determines the diffusion of damage around the crack. The smaller the parameter, the more resolved the crack, however, the size is also mesh-dependent, so be careful. Best practice is to use $l_0=2l_m$, where $l_m$ is the element size in the discretised mesh (the UQ value provided in the report uses $l_0=5l_m$, how does this impact the accuracy?).
- `max_displacement` (**optional**, default=0.06): this is the maximum displacement that the structure will be displaced to.
- `loading_directions` (**optional**, default=[0, 1]): this is the loading directions of the system. Specify the loading in fractions of `max_displacement`. It always needs to start with 0 (as the system starts at rest!). To put it simply, if you want to just load it from 0 to maximum displacement, keep it as [0, 1]. If you want to, for example, load $0\text{mm}\to0.1\text{mm}\to0.05\text{mm}\to0.1\text{mm}$ with a `max_displacement` of 0.1mm, you would specify [0, 1, 0.5, 1]. 
- `domain` (**optional**, default=[0, 100]): specify the domain in terms of mm. As all of the experiments were done with the default domain, I recommend you do not change this.

The outputs you can expect are:
- `forces`: the applied force history array.
- `positions`: the displaced node position history array.
- `degradations`: the degradations at pinned node history array.
- `max_displacement`: the maximum displacement the bar is subjected to.
- `final_pf`: the final phase field profile array.
- `fracture_energies`: the fracture energy history array.

The material parameters for the structure are hardcoded. If you would like to investigate the effect of material parameters, feel free to extend the code in `run_fem.py`, lines 42-55.

## Sample code for batch simulation
For your convenience, I have included some sample code which I used to produce results in batches for mesh convergence analysis, along with the code for producing the plots. Feel free to modify and use it.

In [ ]:
import numpy as np
from run_fem import run_experiment

timesteps = 1000
lengthscale_parameter = 0.5

element_counts = [100, 200, 500, 1000, 2000, 4000, 6000]

results = {}
for element_count in element_counts:
    print(f"Running experiment for {element_count} elements")
    forces, positions, degradations, max_displacement, final_pf, fracture_energies = (
        run_experiment(
            timesteps=timesteps, element_count=element_count, lengthscale_parameter=lengthscale_parameter
        )
    )
    results[element_count] = {
        "force": np.array(forces),
        "positions": np.array(positions),
        "degradations": np.array(degradations),
        "final_pf": np.array(final_pf),
        "fracture_energy": np.array(fracture_energies),
    }

In [ ]:
from matplotlib import pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes, mark_inset
from matplotlib.ticker import FormatStrFormatter

element_counts = list(results.keys())

## Initialise main plot
fig, ax = plt.subplots(1, 1, figsize=(6, 6))

## Produce main plot
for element_count in element_counts:
    ax.plot(results[element_count]["positions"], results[element_count]["force"], label=f"{element_count} elements")

ax.legend()
ax.ticklabel_format(style='sci', axis='x', scilimits=(0,0));
ax.set_xlabel("Displacement, mm");
ax.set_ylabel("Total Applied Force, kN");
ax.set_title(f"Varying number of elements with \n{timesteps} timesteps and $l_0={lengthscale_parameter}$mm")

## Initialise inset
ax_inset = inset_axes(ax, width="40%", height="40%", loc='lower left', borderpad=5)

## Format x-axis
x_domain = np.array([5.02, 5.25]) * 1e-2
ax_inset.set_xlim(x_domain)
ax_inset.set_xticks(np.linspace(x_domain[0], x_domain[1], 3))

## Format y-axis
y_domain = np.array([0.022, 0.02372])
ax_inset.set_ylim(y_domain)
ax_inset.set_yticks(np.linspace(y_domain[0], y_domain[1], 3))

## Format tick labels
ax_inset.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
ax_inset.yaxis.set_major_formatter(FormatStrFormatter('%.4f'))

## Plot inset
for element_count in element_counts:
    ax_inset.plot(results[element_count]["positions"], results[element_count]["force"])

## Mark inset
mark_inset(ax, ax_inset, loc1=2, loc2=4, fc="none", ec="0.5")

## Save the plot
plt.tight_layout()
plt.savefig("figures/qs_1d_varied_mesh_fd.pdf", format="pdf")